In [23]:
import os
import json

def build_label_dict():
    d = {}
    label_path = r"./Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split("\t")
        d[k] = True if v.strip()=="1" else False
    return d

In [24]:
#returns chat dictionary that include the label of each chat, and turns where each turn has sentences list and elapsed 
#time + the speaker id

def gen_chat_data():
    chat_path = r"./Frames-dataset/chats"
    chats = {}
    d = build_label_dict()
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        
        turns = []
        if 'turns' in chat:
            tsp = chat['turns'][0]['timestamp']
       
            for turn in chat['turns']:
               
                ts = turn['timestamp'] - tsp
                tsp = turn['timestamp']
                turns.append({"ti":ts,"text":turn["text"],"author":turn["author"]})
        if filename in d:
            chats[filename] = {}
            chats[filename]["turns"] = turns
            chats[filename]["label"] = d[filename]#[:-5]
    return chats

In [25]:
import numpy as np
# import pandas as pd
import _pickle as cPickle
from collections import defaultdict
import re

from numpy.linalg import norm

import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers.merge import concatenate
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
import theano

import functools
import nltk
nltk.download('punkt')
from nltk import tokenize

MAX_SENT_LENGTH = 40
MAX_SENTS = 20
MAX_NB_WORDS = 20000
MAX_TURNS = 60
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 0.2
RNN_DIM = 25
GLOVE_DIR = "./data/glove"

[nltk_data] Downloading package punkt to /home/amirgilad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
def create_data(chats,l, flag=False,length=0):
    texts = []
    labels=[]
    turns=[]
    for idx in chats.keys():
        if flag:
            text = "\n".join([x["text"] for x in chats[idx]["turns"][:length]])
            
        else:
            text = "\n".join([x["text"] for x in chats[idx]["turns"]])
        texts.append(text)
        sentences = tokenize.sent_tokenize(text)
        turns.append(sentences)
        labels.append(chats[idx]["label"])
        
    tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(texts)

    data = np.zeros((l, MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
    for i, sentences in enumerate(turns):
        if flag and i==length:
            break;
        for j, sent in enumerate(sentences):
            if j< MAX_SENTS:
                wordTokens = text_to_word_sequence(sent)
                k=0
                for _, word in enumerate(wordTokens):
                    if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                        data[i,j,k] = tokenizer.word_index[word]
                        k=k+1   
    return data


In [27]:
def create_texts_labels(chats,flag=False,length=0):
    texts=[]
    labels=[]
    for idx in chats.keys():
        if flag:
        
            text = "\n".join([x["text"] for x in chats[idx]["turns"][:length]])
        else:
            text = "\n".join([x["text"] for x in chats[idx]["turns"]])
        texts.append(text)
        sentences = tokenize.sent_tokenize(text)
        labels.append(chats[idx]["label"])   
    return texts,labels


In [28]:
def prepare_datasets(chats,flag=False, length =0):
    if flag:
        chats = {k: v for k, v in chats.items() if len(v["turns"])>length}
    
    texts,labels = create_texts_labels(chats,flag,length)

    tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(texts)
    data = create_data(chats,len(texts),flag,length)
    word_index = tokenizer.word_index
    print('Total %s unique tokens.' % len(word_index))

    labels = to_categorical(np.asarray(labels))
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)
#     data = data[:13]
#     labels = labels[:13]
    indices = np.arange(len(data))
    np.random.shuffle(indices)
    data = data[indices]
    labels = labels[indices]
    nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
    nb_test_samples = int(TEST_SPLIT * data.shape[0])

    x_train = data[:-(nb_validation_samples+nb_test_samples)]
    y_train = labels[:-(nb_validation_samples+nb_test_samples)]
    x_val = data[-(nb_validation_samples+nb_test_samples):-nb_test_samples]
    y_val = labels[-(nb_validation_samples+nb_test_samples):-nb_test_samples]
    x_test = data[-nb_test_samples:]
    y_test = labels[-nb_test_samples:]
    
    return word_index,x_train, y_train,x_val, y_val,x_test,y_test


In [29]:
def create_embedding_index():
    embeddings_index = {}
    f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return embeddings_index

In [30]:
def create_embedding_matrix(word_index):
    embeddings_index= create_embedding_index();
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))

    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [31]:
import numpy as np
# import pandas as pd
# import cPickle
from collections import defaultdict
import re

# from bs4 import BeautifulSoup

import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


# chats = gen_chat_data()

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

chats = gen_chat_data()#pd.read_csv('~/Testground/data/imdb/labeledTrainData.tsv', sep='\t')
# print (data_train.shape)
word_index,x_train, y_train,x_val, y_val,x_test,y_test = prepare_datasets(chats)
# sequences = tokenizer.texts_to_sequences(y_train)

# word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y_train.shape)


print('Number of positive and negative reviews in traing and validation set ')
print (y_train.sum(axis=0))
print (y_val.sum(axis=0))

embedding_index = create_embedding_index()
embedding_matrix = create_embedding_matrix(word_index)

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - simplified convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=128)

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=20, batch_size=50)

/usr/local/lib/python3.4/dist-packages/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Total 6954 unique tokens.
Shape of data tensor: (1369, 20, 40)
Shape of label tensor: (1369, 2)
Found 6954 unique tokens.
Shape of data tensor: (0, 1000)
Shape of label tensor: (823, 2)
Number of positive and negative reviews in traing and validation set 
[ 318.  505.]
[ 103.  170.]
Total 400000 word vectors.
Total 400000 word vectors.
Total 0 word vectors in Glove 6B 100d.


OSError: [Errno 12] Cannot allocate memory